# ConversationBufferMemory

Hướng dẫn này giới thiệu `ConversationBufferMemory`, một class memory lưu trữ lịch sử hội thoại trong một buffer.

Thông thường, không cần xử lý bổ sung. Tuy nhiên, đôi khi, điều này có thể cần thiết khi lịch sử hội thoại vượt quá context window của model.

Trong hướng dẫn này, chúng ta sẽ học cách sử dụng `ConversationBufferMemory` để lưu trữ và truy xuất lịch sử hội thoại.


In [1]:
# Setup environment
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.globals import set_llm_cache
from langchain_community.cache import SQLiteCache

load_dotenv(override=True, dotenv_path="../.env")

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.0)
prompt = PromptTemplate.from_template("Liệt kê 5 món ăn ngon ở {place}")
chain = prompt | llm | StrOutputParser()

# Create cache directory
if not os.path.exists("cache"):
    os.makedirs("cache")

# Set SQLiteCache
set_llm_cache(SQLiteCache(database_path="cache/llm_cache.db"))

answer = chain.invoke("Hà Nội")
print(answer)

1. Phở Hà Nội: Một trong những món ăn nổi tiếng và phổ biến nhất của Hà Nội, phở Hà Nội có hương vị đặc trưng, ngon và hấp dẫn.

2. Bún chả: Một món ăn truyền thống của Hà Nội, bún chả gồm bún, thịt nướng và nước mắm chua ngọt, tạo nên hương vị đặc trưng và hấp dẫn.

3. Bánh mỳ pate: Một món ăn sáng phổ biến ở Hà Nội, bánh mỳ pate có vị ngon, thơm và béo của pate kết hợp với vị giòn của bánh mỳ.

4. Nem chua rán: Một món ăn vặt phổ biến của người dân Hà Nội, nem chua rán có vị chua, cay, ngọt và giòn rất hấp dẫn.

5. Bún ốc: Một món ăn đặc trưng của Hà Nội, bún ốc có hương vị đậm đà, thơm ngon và hấp dẫn, được nhiều người yêu thích.


## Extracting messages as strings

After storing conversation messages, this memory allows you to extract messages into a variable.

In [2]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory()
memory

/tmp/ipykernel_718416/3227628576.py:2: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory()


ConversationBufferMemory(chat_memory=InMemoryChatMessageHistory(messages=[]))

You can use the `save_context(inputs, outputs)` method to save conversation records.

- This method accepts two arguments, `inputs` and `outputs`.
- `inputs` stores the user's question, and `outputs` stores the AI's answer.
- The conversation record is stored internally under the `history` key.
- You can then use the `load_memory_variables` method to retrieve and inspect the saved conversation history.

In [3]:
# inputs: dictionary(key: "human" or "ai", value: question)
# outputs: dictionary(key: "ai" or "human", value: answer)
memory.save_context(
    inputs={
        "human": "Hello, I want to open a bank account remotely. How do I start?",
    },
    outputs={
        "ai": "Hello! I'm glad you want to open an account. First, please prepare your ID for identity verification."
    },
)

In [4]:
memory

ConversationBufferMemory(chat_memory=InMemoryChatMessageHistory(messages=[HumanMessage(content='Hello, I want to open a bank account remotely. How do I start?', additional_kwargs={}, response_metadata={}), AIMessage(content="Hello! I'm glad you want to open an account. First, please prepare your ID for identity verification.", additional_kwargs={}, response_metadata={})]))

The `load_memory_variables({})` method of the memory object returns the complete message history as a string.

In [5]:
# Check the message history stored in the 'history' key.
print(memory.load_memory_variables({})["history"])

Human: Hello, I want to open a bank account remotely. How do I start?
AI: Hello! I'm glad you want to open an account. First, please prepare your ID for identity verification.


In [6]:
memory.save_context(
    inputs={
        "human": "Yes, I've prepared my ID for identity verification. What should I do next?"
    },
    outputs={
        "ai": "Thank you. Please upload the front and back of your ID clearly. We will proceed with the identity verification process next."
    },
)

In [7]:
# Check the message history stored in the 'history' key.
print(memory.load_memory_variables({})["history"])

Human: Hello, I want to open a bank account remotely. How do I start?
AI: Hello! I'm glad you want to open an account. First, please prepare your ID for identity verification.
Human: Yes, I've prepared my ID for identity verification. What should I do next?
AI: Thank you. Please upload the front and back of your ID clearly. We will proceed with the identity verification process next.


In [8]:
# Save 2 conversations.
memory.save_context(
    inputs={
        "human": "I uploaded the photo. How do I proceed with identity verification?"
    },
    outputs={
        "ai": "We have confirmed the photo you uploaded. Please proceed with identity verification through your mobile phone. Please enter the verification number sent by text."
    },
)
memory.save_context(
    inputs={
        "human": "I entered the verification number. How do I open an account now?"
    },
    outputs={
        "ai": "Identity verification has been completed. Please select the type of account you want and enter the necessary information. You can choose the type of deposit, currency, etc."
    },
)

In [9]:
# Check the conversation history stored in the 'history' key.
print(memory.load_memory_variables({})["history"])

Human: Hello, I want to open a bank account remotely. How do I start?
AI: Hello! I'm glad you want to open an account. First, please prepare your ID for identity verification.
Human: Yes, I've prepared my ID for identity verification. What should I do next?
AI: Thank you. Please upload the front and back of your ID clearly. We will proceed with the identity verification process next.
Human: I uploaded the photo. How do I proceed with identity verification?
AI: We have confirmed the photo you uploaded. Please proceed with identity verification through your mobile phone. Please enter the verification number sent by text.
Human: I entered the verification number. How do I open an account now?
AI: Identity verification has been completed. Please select the type of account you want and enter the necessary information. You can choose the type of deposit, currency, etc.


In [10]:
# Save 2 more conversations.
memory.save_context(
    inputs={
        "human": "I've entered all the information. What's the next step?",
    },
    outputs={
        "ai": "I've confirmed the information you've entered. The account opening process is almost complete. Please agree to the terms of use and confirm the account opening."
    },
)
memory.save_context(
    inputs={
        "human": "I've completed all the steps. Has the account been opened?",
    },
    outputs={
        "ai": "Yes, the account has been opened. Your account number and related information have been sent to the email you registered. If you need additional help, please contact us at any time. Thank you!"
    },
)

In [11]:
# Check the conversation history stored in the 'history' key.
print(memory.load_memory_variables({})["history"])

Human: Hello, I want to open a bank account remotely. How do I start?
AI: Hello! I'm glad you want to open an account. First, please prepare your ID for identity verification.
Human: Yes, I've prepared my ID for identity verification. What should I do next?
AI: Thank you. Please upload the front and back of your ID clearly. We will proceed with the identity verification process next.
Human: I uploaded the photo. How do I proceed with identity verification?
AI: We have confirmed the photo you uploaded. Please proceed with identity verification through your mobile phone. Please enter the verification number sent by text.
Human: I entered the verification number. How do I open an account now?
AI: Identity verification has been completed. Please select the type of account you want and enter the necessary information. You can choose the type of deposit, currency, etc.
Human: I've entered all the information. What's the next step?
AI: I've confirmed the information you've entered. The accoun

## Extracting messages as `HumanMessage` and `AIMessage` objects

Setting `return_messages=True` returns `HumanMessage` and `AIMessage` objects.

In [12]:
memory = ConversationBufferMemory(return_messages=True)

memory.save_context(
    inputs={
        "human": "Hello, I want to open a bank account remotely. How do I start?",
    },
    outputs={
        "ai": "Hello! I'm glad you want to open an account. First, please prepare your ID for identity verification.",
    },
)

memory.save_context(
    inputs={
        "human": "Yes, I've prepared my ID for identity verification. What should I do next?"
    },
    outputs={
        "ai": "Thank you. Please upload the front and back of your ID clearly. We will proceed with the identity verification process next."
    },
)

memory.save_context(
    inputs={
        "human": "I uploaded the photo. How do I proceed with identity verification?"
    },
    outputs={
        "ai": "We have confirmed the photo you uploaded. Please proceed with identity verification through your mobile phone. Please enter the verification number sent by text."
    },
)

In [13]:
# Check the conversation history stored in the 'history' key.
memory.load_memory_variables({})["history"]

[HumanMessage(content='Hello, I want to open a bank account remotely. How do I start?', additional_kwargs={}, response_metadata={}),
 AIMessage(content="Hello! I'm glad you want to open an account. First, please prepare your ID for identity verification.", additional_kwargs={}, response_metadata={}),
 HumanMessage(content="Yes, I've prepared my ID for identity verification. What should I do next?", additional_kwargs={}, response_metadata={}),
 AIMessage(content='Thank you. Please upload the front and back of your ID clearly. We will proceed with the identity verification process next.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I uploaded the photo. How do I proceed with identity verification?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='We have confirmed the photo you uploaded. Please proceed with identity verification through your mobile phone. Please enter the verification number sent by text.', additional_kwargs={}, response_metadata=

## Applying to a Chain

Let's apply `ConversationBufferMemory` to a `ConversationChain`.

In [15]:
from langchain.chains import ConversationChain

conversation = ConversationChain(
  llm=llm,
  memory=ConversationBufferMemory()
)

/tmp/ipykernel_718416/675096263.py:3: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :meth:`~RunnableWithMessageHistory: https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html` instead.
  conversation = ConversationChain(


In [17]:
response = conversation.invoke("Liệt kê 5 món ăn ngon ở Hà Nội")
print(response)

{'input': 'Liệt kê 5 món ăn ngon ở Hà Nội', 'history': 'Human: Liệt kê 5 món ăn ngon ở Hà Nội\nAI: 1. Phở - a traditional Vietnamese noodle soup with flavorful broth, rice noodles, herbs, and your choice of meat.\n2. Bún chả - grilled pork served with rice noodles, herbs, and dipping sauce.\n3. Bánh mì - a Vietnamese sandwich with a crispy baguette filled with various meats, pickled vegetables, and herbs.\n4. Bún bò Nam Bộ - a noodle dish with marinated beef, fresh herbs, peanuts, and a tangy dressing.\n5. Nem cua bể - fried crab spring rolls, a popular street food in Hanoi.', 'response': 'Xin lỗi, tôi không biết.'}


In [18]:
# Send a request to summarize the previous conversation in bullet points.
response = conversation.invoke("Summarize the previous answer in bullet points.")
print(response)

{'input': 'Summarize the previous answer in bullet points.', 'history': 'Human: Liệt kê 5 món ăn ngon ở Hà Nội\nAI: 1. Phở - a traditional Vietnamese noodle soup with flavorful broth, rice noodles, herbs, and your choice of meat.\n2. Bún chả - grilled pork served with rice noodles, herbs, and dipping sauce.\n3. Bánh mì - a Vietnamese sandwich with a crispy baguette filled with various meats, pickled vegetables, and herbs.\n4. Bún bò Nam Bộ - a noodle dish with marinated beef, fresh herbs, peanuts, and a tangy dressing.\n5. Nem cua bể - fried crab spring rolls, a popular street food in Hanoi.\nHuman: Liệt kê 5 món ăn ngon ở Hà Nội\nAI: Xin lỗi, tôi không biết.', 'response': '- Phở: traditional Vietnamese noodle soup with flavorful broth, rice noodles, herbs, and choice of meat\n- Bún chả: grilled pork served with rice noodles, herbs, and dipping sauce\n- Bánh mì: Vietnamese sandwich with crispy baguette filled with meats, pickled vegetables, and herbs\n- Bún bò Nam Bộ: noodle dish with 